In [3]:
import os
import sys
import numpy as np
import numpy.matlib as npm
import utm
# import ast
import copy
import math
import scipy
import pandas as pd
from scipy.stats import norm
from scipy.stats import distributions
# from operator import itemgetter
# from numba import jit
import cartopy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
from datetime import datetime, timezone, timedelta
import json
import requests


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371 # Radius of earth in kilometers.
        return c * r

In [ ]:
def map_setup():
    proj = cartopy.crs.PlateCarree()
    fig = plt.figure(figsize=(16, 8))

    # ax = plt.axes(projection=cartopy.crs.Mercator())
    usemap_proj = cartopy.crs.PlateCarree(central_longitude=180)
    ax = plt.axes(projection=usemap_proj)

    coastline = cartopy.feature.GSHHSFeature(scale='low', levels=[1])
    # coastline = cartopy.feature.GSHHSFeature(scale='high', levels=[1])
    ax.add_feature(coastline, edgecolor='#000000', facecolor='#cccccc', linewidth=1)
    ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'))
    ax.add_feature(cartopy.feature.STATES.with_scale('50m'))
    ax.add_feature(cartopy.feature.OCEAN.with_scale('50m'))
    gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1,
                        color="#ffffff", alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True
    gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
    gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}

    return ax, proj


def plot_pois(pois_coords, ind, event_dict):

    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']

    ax, proj = map_setup()
    
    ax.plot(pois_coords[ind,0], pois_coords[ind,1], markerfacecolor="#0ba518", marker="o",
                linewidth=0, markeredgecolor="#000000",
                transform=proj)#, zorder=10)

    ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=10,
            markerfacecolor='red', markeredgecolor='#000000',
            transform=proj)
    
    # ax.set_extent([19., 30.5, 35.5, 41.5], crs=proj)

    ax.set_xlabel(r'Longitude ($^\circ$)', fontsize=14)
    ax.set_ylabel(r'Latitude ($^\circ$)', fontsize=14)

def plot_barycenters(bs_coords, prob, event_dict):

    # fig,axs = plt.subplots(2,1,figsize=(20,20),subplot_kw={'projection': cartopy.crs.Mercator()})
    # fig.set_tight_layout(True)
    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']

    ax, proj = map_setup()
 
    lonlat_list=[list(x) for x in set(tuple(x) for x in bs_coords)]
    nloc=len(lonlat_list)
 
    sumprob=[]
    for iloc in range(nloc):
        locxy=lonlat_list[iloc]
        isel = [i for i, x in enumerate(list(bs_coords)) if np.allclose(x, locxy)]
        # isel = [i for i, x in enumerate(list(bs_coords)) if x == locxy]
        #print(len(isel))
        tmplist=[prob[i] for i in isel]
        sumprob.append(sum(tmplist))
    lon = [el[0] for el in lonlat_list]
    lat = [el[1] for el in lonlat_list]
 
    # lon = bs_coords[:,0] 
    # lat = bs_coords[:,1]
    cmap = ax.scatter(lon,lat,s=30, transform=proj,edgecolors='k',c=sumprob,cmap=plt.cm.plasma)#, label=labels[ic], c=colors[ic])
    cbar = plt.colorbar(cmap,ax=ax,extend='both',extendfrac='auto',aspect=30,format='%.0e',pad=0.02,
                 label='Cumulated Probability (for each barycenter)',shrink=0.8)
 
    ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=10,
            markerfacecolor='red', markeredgecolor='#000000',
            transform=proj)
     
    ax.set_extent([140, 150, 35, 45], crs=proj)

    cbar.locator = ticker.MaxNLocator(nbins=10)
    cbar.update_ticks()

In [ ]:
# These functions are copied from the ptf_pre_load.py file

def load_intensity_thresholds(data_folder):
    """
    READ THRESHOLDS
    """
    
    intensity_thresholds = os.path.join(data_folder, 'intensity_thresholds.npy')

    ith = np.load(intensity_thresholds, allow_pickle=True)
    intensity_measure = ith.item().keys()
    thresholds = ith.item()[list(intensity_measure)[0]]

    return thresholds, intensity_measure


In [ ]:
# These functions are copied from the ptf_scaling_laws.py file
def scalinglaw_WC(**kwargs):
    '''
    Scaling law from Wells&Coppersmith (1994)
    '''

    mag        = kwargs.get('mag', None)
    type_scala = kwargs.get('type_scala', None)

    if (type_scala == 'M2L'):
        a =-2.440
        b =0.590
        y = 10.**(a+b*mag)

    elif (type_scala == 'M2W'):
        a=-1.010
        b=0.320
        y = 10.**(a+b*mag)

    else:
        raise Exception("Scaling law in scalinglaw_WC not recognized. Exit!")
        #print("Scaling law in scalinglaw_WC not recognized. Exit!")
        #sys.exit()

    return y

def scalinglaw_Murotani(**kwargs):
    '''
    Scaling law from Murotani et al (2013)
    '''

    mag        = kwargs.get('mag', None)
    type_scala = kwargs.get('type_scala', None)

    a    = -3.806
    b    = 1.000
    Area = 10**(a+b*mag)

    if (type_scala == 'M2L'):
        y = math.sqrt(2.5*Area)/2.5

    elif (type_scala == 'M2W'):
        y = math.sqrt(2.5*Area)

    else:
        raise Exception("Scaling law in scalinglaw_Murotani not recognized. Exit!")
        #print("Scaling law in scalinglaw_Murotani not recognized. Exit!")
        #sys.exit()

    return y

def mag_to_l_BS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_WC(mag=mag, type_scala='M2L')

    return out

def mag_to_w_BS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_WC(mag=mag, type_scala='M2W')

    return out

def mag_to_l_PS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_Murotani(mag=mag, type_scala='M2W')

    return out

def correct_BS_horizontal_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = 0.5 * mag_to_l_BS(mag=mag)

    return out

def correct_PS_horizontal_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = 0.5 * mag_to_l_PS(mag=mag)

    return out
 
def correct_BS_vertical_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = math.sin(math.pi/4)*0.5 * mag_to_w_BS(mag=mag)

    return out

In [ ]:
# These functions are copied from the ptf_mix_utilities.py file

def NormMultiDvec(**kwargs):

    """
    # Here mu and sigma, already inserted into ee dictionary
    # Coordinates in utm
    mu = tmpmu =PosMean_3D = [EarlyEst.lonUTM,EarlyEst.latUTM,EarlyEst.Dep*1.E3]
    Sigma = tmpCOV = EarlyEst.PosCovMat_3D = [EarlyEst.PosSigmaXX EarlyEst.PosSigmaXY EarlyEst.PosSigmaXZ; ...
                         EarlyEst.PosSigmaXY EarlyEst.PosSigmaYY EarlyEst.PosSigmaYZ; ...
                         EarlyEst.PosSigmaXZ EarlyEst.PosSigmaYZ EarlyEst.PosSigmaZZ];
    mu =     np.array([ee['lon'], ee['lat'], ee['depth']*1000.0])
    sigma =  np.array([[ee['cov_matrix']['XX'], ee['cov_matrix']['XY'], ee['cov_matrix']['XZ']], \
                       [ee['cov_matrix']['XY'], ee['cov_matrix']['YY'], ee['cov_matrix']['YZ']], \
                       [ee['cov_matrix']['XZ'], ee['cov_matrix']['YZ'], ee['cov_matrix']['ZZ']]])
    """

    x     = kwargs.get('x', None)
    mu    = kwargs.get('mu', None)
    sigma = kwargs.get('sigma', None)

    n = len(mu)

    #mu = np.reshape(mu,(3,1))
    mu = np.reshape(mu,(n,1))
    t1  = (2 * math.pi)**(-1*len(mu)/2)
    t2  = 1 / math.sqrt(np.linalg.det(sigma))
    #c1  = npm.repmat(mu, 1, np.shape(mu)[0])
    c1  = npm.repmat(mu, 1, len(x))
    c11 = (x - c1.transpose()).transpose()
    c12 = x - c1.transpose()

    d  = np.linalg.lstsq(sigma, c11, rcond=None)[0]
    e = np.dot(c12, d)
    f = np.multiply(-0.5,np.diag(e))
    g = np.exp(f)
    h = t1 * t2 * g

    return h

In [4]:
# These functions are copied from the ptf_load_event.py file
def int_quake_cat2dict(json_object):

    d = dict()

    # Event Ids
    try:
        origin_id =  str(json_object['features'][0]['properties']['originid'])
    except:
        origin_id =  str(json_object['features'][0]['properties']['originId'])

    event_id      =  str(json_object['features'][0]['properties']['eventId'])
    # author        =  str(json_object['features'][0]['properties']['author'])
    # version       =  str(json_object['features'][0]['properties']['version'])

    # Epicenter informations
    lon           =  float(json_object['features'][0]['geometry']['coordinates'][0])
    lat           =  float(json_object['features'][0]['geometry']['coordinates'][1])
    depth         =  float(json_object['features'][0]['geometry']['coordinates'][2])
    OT            =  str(json_object['features'][0]['properties']['time'])
    ev_type       =  str(json_object['features'][0]['properties']['type'])
    mag_type      =  str(json_object['features'][0]['properties']['magType'])
    place          =  str(json_object['features'][0]['properties']['place'])

    #utm conversion
    ee_utm = utm.from_latlon(lat, lon)

    # Specific Mag percentiles and covariant cov_matrix
    mag_percentiles = json_object['features'][0]['properties']['mag_percentiles']
    cov_matrix      = json_object['features'][0]['properties']['cov_matrix']
    
    pos_Sigma = cov_matrix.copy() #json_string['features'][0]['properties']['cov_matrix']

    cov_matrix['XX'] = float(cov_matrix['XX'])
    cov_matrix['XY'] = float(cov_matrix['XY'])
    cov_matrix['XZ'] = float(cov_matrix['XZ'])
    cov_matrix['YY'] = float(cov_matrix['YY'])
    cov_matrix['YZ'] = float(cov_matrix['YZ'])
    cov_matrix['ZZ'] = float(cov_matrix['ZZ'])

    pos_Sigma['XX']  = float(pos_Sigma['XX']) * 1e6
    pos_Sigma['XY']  = float(pos_Sigma['XY']) * 1e6
    pos_Sigma['XZ']  = float(pos_Sigma['XZ']) * 1e6
    pos_Sigma['YY']  = float(pos_Sigma['YY']) * 1e6
    pos_Sigma['YZ']  = float(pos_Sigma['YZ']) * 1e6
    pos_Sigma['ZZ']  = float(pos_Sigma['ZZ']) * 1e6

    mag_percentiles['p16']  = float(mag_percentiles['p16'])
    mag_percentiles['p50']  = float(mag_percentiles['p50'])
    mag_percentiles['p84']  = float(mag_percentiles['p84'])
    mag_sigma = 0.5 * (mag_percentiles['p84'] - mag_percentiles['p16'])

    d['eventid']        = event_id
    d['originid']       = origin_id
    d['lat']            = lat
    d['lon']            = lon
    d['depth']          = depth
    d['ot']             = OT
    d['mag']            = mag_percentiles['p50']
    d['mag_percentiles'] = mag_percentiles
    d['MagSigma']        = mag_sigma
    d['type']           = ev_type
    d['mag_type']       = mag_type
    d['ee_utm']         = ee_utm
    d['place']          = place

    # d['version']        = "%03d" % (float(version))
    # d['author']         = author
    # d['area']           = area
    # d['area_geo']       = area_geo
    # d['mag']            = mag
    # d['mag_values']     = mag_values
    # d['mag_counts']     = mag_counts
    # d['ct']             = creation_time
    # d['ot_year']        = origin_year
    # d['ot_month']       = origin_month
    # d['ot_day']         = origin_day

    d['cov_matrix']      = cov_matrix
    d['pos_Sigma']       = pos_Sigma

    d['ee_PosCovMat_2d'] = np.array([[cov_matrix['XX'], cov_matrix['XY']], \
                                     [cov_matrix['XY'], cov_matrix['YY']]])
    d['PosMean_2d']      = np.array([d['ee_utm'][0], \
                                     d['ee_utm'][1]])
    d['PosCovMat_3d']    = np.array([[cov_matrix['XX'], cov_matrix['XY'], cov_matrix['XZ']], \
                                     [cov_matrix['XY'], cov_matrix['YY'], cov_matrix['YZ']], \
                                     [cov_matrix['XZ'], cov_matrix['YZ'], cov_matrix['ZZ']]])
    d['PosCovMat_3dm']    = d['PosCovMat_3d']*1000000
    d['PosMean_3d']      = np.array([d['ee_utm'][0], \
                                     d['ee_utm'][1], \
                                     d['depth'] * 1000.0])

    # d['root_name']       = str(d['ot_year']) + str(d['ot_month']) + str(d['ot_day']) + '_' + \
    #                        d['area']
    #                        #str(d['eventid']) + '_' + str(d['version']) + '_' + d['area']

    return d

def compute_position_sigma_lat_lon(event_parameters):
    """
    REFERENCE LAT = YY
    REFERENCE LON = XX
    """

    bs_mag_max          = 8.1

    sigma               = event_parameters['sigma']
    event_mag           = event_parameters['mag_percentiles']['p50']
    event_mag_max       = event_parameters['mag_percentiles']['p50'] + \
                          event_parameters['MagSigma'] * sigma
    event_mag_sigma     = event_parameters['MagSigma']

    event_cov_xx        = event_parameters['pos_Sigma']['XX']
    event_cov_xy        = event_parameters['pos_Sigma']['XY']
    event_cov_yy        = event_parameters['pos_Sigma']['YY']


    mag_to_correct      = min(bs_mag_max, event_mag_max)

    delta_position_BS_h = correct_BS_horizontal_position(mag=mag_to_correct)
    delta_position_PS_h = correct_PS_horizontal_position(mag=event_mag + sigma * event_mag_sigma)

    position_BS_sigma_yy  = math.sqrt(abs(event_cov_yy)) + delta_position_BS_h
    position_BS_sigma_xx  = math.sqrt(abs(event_cov_xx)) + delta_position_BS_h
    position_BS_sigma_xy  = math.sqrt(abs(event_cov_xy)) + delta_position_BS_h

    event_parameters['position_BS_sigma_yy'] = position_BS_sigma_yy
    event_parameters['position_BS_sigma_xx'] = position_BS_sigma_xx
    event_parameters['position_BS_sigma_xy'] = position_BS_sigma_xy

    position_PS_sigma_yy  = math.sqrt(abs(event_cov_yy)) + delta_position_PS_h
    position_PS_sigma_xx  = math.sqrt(abs(event_cov_xx)) + delta_position_PS_h
    position_PS_sigma_xy  = math.sqrt(abs(event_cov_xy)) + delta_position_PS_h

    event_parameters['position_PS_sigma_yy'] = position_PS_sigma_yy
    event_parameters['position_PS_sigma_xx'] = position_PS_sigma_xx
    event_parameters['position_PS_sigma_xy'] = position_PS_sigma_xy

    return event_parameters


In [ ]:
# These functions are copied from the ptf_define_ensemble_global.py file
def select_magnitude(event_parameters, magnitude_values, sigma):
    """
    Selecting magnitude values to create the ensemble.
    If magnitude distribution is provided in json file then the values 
    in the key [mag_values] are used. 
    Otherwise, magnitude ranges are defined from the uncertainty provided 
    in the key parameter [mag_percentiles]
    
    """
    print('--> Selecting magnitude values')

    min_mag  = event_parameters['mag'] - event_parameters['MagSigma'] * sigma
    max_mag  = event_parameters['mag'] + event_parameters['MagSigma'] * sigma

    magnitudes = magnitude_values[(magnitude_values >= min_mag) & (magnitude_values <= max_mag)]
    idx = np.where((magnitude_values >= min_mag) & (magnitude_values <= max_mag))
    if magnitudes.size == 0:
        idx = np.array((np.abs(magnitude_values-max_mag)).argmin())
        magnitudes = np.array([magnitude_values[idx]])

    print('    Number of magnitude values = ' + str(len(magnitudes)))
    print('    Magnitudes selected: ' + str(magnitudes))

    return magnitudes, idx

def discretized_position(ellipse, step):

    print('--> Discretizing positions')

    utm_x_min = np.amin(ellipse[:,1])
    utm_x_max = np.amax(ellipse[:,1])
    utm_y_min = np.amin(ellipse[:,0])
    utm_y_max = np.amax(ellipse[:,0])
    utm_x = np.arange(utm_x_min, utm_x_max + step, step)
    utm_y = np.arange(utm_y_min, utm_y_max + step, step)
    
    print('    Number of points along x= ' + str(len(utm_x)))
    print('    Number of points along y= ' + str(len(utm_y)))

    return utm_x, utm_y

def discretized_depth(z, step):

    print('--> Discretizing depths')

    min_depth = np.amin(z)
    max_depth = np.amax(z)

    depth = np.arange(min_depth, max_depth + step, step)
    print('    Number of points along z= ' + str(len(depth)))
    
    return depth

def create_grid3d(event_parameters, ensemble): 
    """
    Creation of grid 3d of position in utm (x, y, z) and conversion to geographic coordinates (lon, lat).
    In ensemble['grid_3d'] points are ordered as follows: x1,y1,z1; x1,y2,z1; ...; x2,y1,z1;...
    """

    print('--> Creating grid 3d')

    zone_number = event_parameters['ee_utm'][2]
    zone_letter = event_parameters['ee_utm'][3]

    position_x = ensemble['position_utm_x']
    position_y = ensemble['position_utm_y']
    depth      = ensemble['depth']

    xx_3d, yy_3d, zz_3d = np.meshgrid(position_x, position_y, depth, indexing='xy')
    xx_3d               = xx_3d.flatten('F')
    yy_3d               = yy_3d.flatten('F')
    zz_3d               = zz_3d.flatten('F')
    grid_3d             = np.array([xx_3d, yy_3d, zz_3d]).transpose()

    # geo_coord = np.array(utm.to_latlon(grid_3d[:,0], grid_3d[:,1], zone_number, zone_letter))
    easting = copy.deepcopy(grid_3d[:,0])
    northing = copy.deepcopy(grid_3d[:,1])
    geo_coord = np.array(utm.to_latlon(easting, northing, zone_number, zone_letter))

    # print(geo_coord.shape, grid_3d.shape)
    print('    Number of grid points = ' + str(grid_3d.shape[0]))
    
    return grid_3d, geo_coord[1,:], geo_coord[0,:]

def discretized_mechanism(fm, stk_step, stk_sigma, dip_step, dip_sigma, rake_step, rake_sigma):
    
    print('--> Discretizing strike, dip, and rake values')

    #discretizing strike angle
    stk_tmp = fm['np1']['strike']
    stk1 = np.arange(stk_tmp - stk_sigma, stk_tmp + stk_sigma + 1, stk_step)
    ind1 = np.argwhere(stk1 < 0)
    stk1[ind1] = 180. - stk1[ind1]
    stk_tmp = fm['np2']['strike']
    stk2 = np.arange(stk_tmp - stk_sigma, stk_tmp + stk_sigma + 1, stk_step)
    ind2 = np.argwhere(stk2 < 0)
    stk2[ind2] = 180. - stk2[ind2]

    #discretizing dip angle
    #TODO BE CAREFUL TO NEGATIVE VALUES OF DIP
    dip_tmp = fm['np1']['dip']
    dip1 = np.arange(dip_tmp - dip_sigma, dip_tmp + dip_sigma + 1, dip_step)
    dip_tmp = fm['np2']['dip']
    dip2 = np.arange(dip_tmp - dip_sigma, dip_tmp + dip_sigma + 1, dip_step)

    #discretizing rake angle
    rake_tmp = fm['np1']['rake']
    rake1 = np.arange(rake_tmp - rake_sigma, rake_tmp + rake_sigma + 1, rake_step)
    rake_tmp = fm['np2']['rake']
    rake2 = np.arange(rake_tmp - rake_sigma, rake_tmp + rake_sigma + 1, rake_step)

    fm1 = np.array(np.meshgrid(stk1, dip1, rake1)).T.reshape(-1,3)
    fm2 = np.array(np.meshgrid(stk2, dip2, rake2)).T.reshape(-1,3)
    focal_mechanism = np.concatenate((fm1, fm2))
    print('    Number of angle combinations = ' + str(focal_mechanism.shape[0]))

    return focal_mechanism

def compute_fault_size(magnitude):
    """
    Fault dimensions (length, width and area) are converted from km to m.
    """
    #TODO understand which scaling law to use in global

    print('--> Computing fault size')

    length = scalinglaw_WC(mag=magnitude, type_scala='M2L')
    width = scalinglaw_WC(mag=magnitude, type_scala='M2W')
    area = length * width
    fault_size = np.vstack((length*1.e3, width*1.e3, area*1.e6)).T
   
    return fault_size

def compute_slip(magnitude, fault_size, mu):
    """
    Calculation of slip from magnitude by scalar seismic moment.

    Scalar seismic moment: M0 = 10**(1.5*(magnitudo+10.7) 
                           Kanamori formula 1977 in dyne⋅cm (10−7 N⋅m)
    Slip on fault        : D(m) = M0(Pa*m3) / area(m2)*mu(Pa)

    """
    print('--> Computing slip')

    area = fault_size[:,0]*fault_size[:,1]
    scalar_moment = 10.**(1.5*(magnitude+10.7)) * 1.e-7
    
    return scalar_moment/(area*mu)

def compute_mag_probability(idx, event_parameters, mag_discretization):
    """
    """

    ev_mag_sigma = event_parameters['MagSigma']
    ev_mag       = event_parameters['mag']

    print('--> Computing magnitude cumulative distribution')

    a = mag_discretization[0:-1]
    b = mag_discretization[1:]
    c = np.add(a, b) * 0.5

    lower = np.insert(c, 0, -np.inf)
    upper = np.insert(c, c.size, np.inf)

    lower_probility_norm  = norm.cdf(lower, ev_mag, ev_mag_sigma)
    upper_probility_norm  = norm.cdf(upper, ev_mag, ev_mag_sigma)

    magnitude_probability = np.subtract(upper_probility_norm, lower_probility_norm)

    magnitude_probability = magnitude_probability[idx]

    return magnitude_probability

def compute_pos_probability(event_parameters, ensemble):
    """
    """

    print('--> Computing position probabilities')

    magnitudes = ensemble['magnitude']
    grid_3d    = ensemble['grid_3d']

    mu = event_parameters['PosMean_3d']

    n_points = grid_3d.shape[0]
    n_mag = len(magnitudes)
    position_probability = np.zeros((n_mag, n_points))
    for imag,mag in enumerate(magnitudes):
        # Compute vertical half_width with respect the magnitude
        v_hwidth = correct_BS_vertical_position(mag = mag)
        h_hwidth = correct_BS_horizontal_position(mag = mag)

        co = copy.deepcopy(event_parameters['PosCovMat_3dm'])
        # Correct  Covariance matrix
        co[0,0] = co[0,0] + h_hwidth**2
        co[1,1] = co[1,1] + h_hwidth**2
        co[2,2] = co[2,2] + v_hwidth**2
        tmp_prob_points = NormMultiDvec(x = grid_3d, mu = mu, sigma = co)
        normfact = np.sum(tmp_prob_points)
        #TODO normalization for each mag (or normalize once outside of the loop?)
        position_probability[imag] = tmp_prob_points / normfact   
   
    return position_probability

def compute_mech_probability(ensemble):

    print('--> Computing focal mechanism probabilities')
    
    # TODO EQUIPROBABILITY FOR ALL MECHANISMS
    n_scenarios, _ = ensemble['focal_mechanism'].shape
    mechanism_probabilities = np.ones((n_scenarios))/n_scenarios              

    return mechanism_probabilities

def scenarios_parameters_and_probabilities(ensemble_probs, ensemble):
    """
    ordered list of scenarios parameters for t-hysea simulations:
       index, mag, lon, lat, depth, strike, dip, rake, length, area, slip
       nparams: number of parameters (assigned manually as 11)

    """

    print('--> Computing the total probability for each scenario')

    n_mag = len(ensemble['magnitude'])
    n_points = ensemble['grid_3d'].shape[0]
    n_foc_mech = ensemble['focal_mechanism'].shape[0]
    #print(n_mag, n_points, n_foc_mech)
    nscen = n_mag * n_points * n_foc_mech
    print('Total number of scenarios = ' + str(nscen))

    nparams = 12
    scen_params = np.zeros((nscen, nparams))
    scen_probs = np.zeros((nscen,3))
    
    iscen = 0
    for imag in range(n_mag):
        for ipoint in range(n_points):
            for ifoc in range(n_foc_mech):
                scen_params[iscen,0] = iscen + 1
                scen_params[iscen,1] = 9999
                scen_params[iscen,2] = ensemble['magnitude'][imag]
                scen_params[iscen,3] = ensemble['position_geo_lon'][ipoint]
                scen_params[iscen,4] = ensemble['position_geo_lat'][ipoint]
                scen_params[iscen,5] = ensemble['grid_3d'][ipoint,2] / 1.e3    # m --> km
                scen_params[iscen,6] = ensemble['focal_mechanism'][ifoc,0]
                scen_params[iscen,7] = ensemble['focal_mechanism'][ifoc,1]
                scen_params[iscen,8] = ensemble['focal_mechanism'][ifoc,2]
                scen_params[iscen,9] = ensemble['fault_size'][imag,0] / 1.e3   # m --> km
                scen_params[iscen,10] = ensemble['fault_size'][imag,2] / 1.e6  # m2 --> km2
                scen_params[iscen,11] = ensemble['slip'][imag]

                scen_probs[iscen,0] = ensemble_probs['magnitude'][imag]
                scen_probs[iscen,1] = ensemble_probs['position'][imag,ipoint]
                scen_probs[iscen,2] = ensemble_probs['focal_mechanism'][ifoc]

                iscen += 1

    scenario_parameters = scen_params
    scenario_probability_pre_norm = scen_probs.prod(axis=1)

    # print('--> Normalizing scenario probabilities')
    #both prob_mag and prob_pos are normalized, normfact=1, maybe not needed!
    normfact = np.sum(scenario_probability_pre_norm) 
    scenario_probability = scenario_probability_pre_norm / normfact
    
    # print(np.sort(scenario_probability)) 
    
    return scenario_parameters, scenario_probability


In [ ]:
# These functions are copied from the ptf_ellipsoid.py file

def build_location_ellipsoid_objects(**kwargs):
    """
    From ellipsedata.m
    % Copyright (c) 2014, Hugo Gabriel Eyherabide, Department of Mathematics
    % and Statistics, Department of Computer Science and Helsinki Institute
    % for Information Technology, University of Helsinki, Finland.
    % All rights reserved.

    !!!! Difference with the original matlab function !!!!
    sigma in this python function is a float
    sigma in matlab is a vector

    """

    ee                 = kwargs.get('event', 'None')
    sigma              = kwargs.get('sigma', 'None')
    seismicity_type    = kwargs.get('seismicity_type', 'None')
   
    # Number of points to set the 2d ellipse
    nr_points = 1000
    
    sigma = float(sigma)

    # 2d Covariant matrix, eigenvalues and eignevectors
    if(seismicity_type == 'BS'):
       cov_matrix   = np.array([ [ee['position_BS_sigma_yy']**2, 0], [0, ee['position_BS_sigma_xx']**2] ])
    elif(seismicity_type == 'PS'):
       cov_matrix   = np.array([ [ee['position_PS_sigma_yy']**2, 0], [0, ee['position_PS_sigma_xx']**2] ])
    else:
       raise Exception('No seismicity type found. Exit')
       #sys.exit('No seismicity type found. Exit')

    # Center of the ellipse
    center = (ee['ee_utm'][1],ee['ee_utm'][0])

    PV, PD = np.linalg.eigh(cov_matrix)
    PV = np.sqrt(np.diag(PV))

    # Build points of ellipse
    theta = np.linspace(0,2*np.pi,nr_points)
    elpt  = np.dot(np.transpose(np.array([np.cos(theta), np.sin(theta)])) , PV)
    elpt  = np.dot(elpt, np.transpose(PD))

    # Add uncertainty
    elpt = elpt * sigma

    # shift to the center
    elpt    = np.transpose(elpt)
    elpt[0] = elpt[0] + center[0]
    elpt[1] = elpt[1] + center[1]
    elpt    = np.transpose(elpt)

    return elpt

In [ ]:
# These functions are copied from the ptf_lambda_bsps_load.py
def load_lambda_BSPS(sigma, ee_d):

    d = dict()

    ## Variables for lambda
    d['lambdaBSPS']                                = {}
    d['lambdaBSPS']['hypo_utm']                    = np.array([ee_d['ee_utm'][0], \
                                                               ee_d['ee_utm'][1], \
                                                               ee_d['depth'] ])
    d['lambdaBSPS']['utmzone_hypo']                = ee_d['ee_utm'][2]
    d['lambdaBSPS']['NormCov']                     = ee_d['PosCovMat_3d']
    d['lambdaBSPS']['confid_lev']                  = norm.cdf(sigma) - norm.cdf(-1 * sigma)
    d['lambdaBSPS']['dchi2']                       = distributions.chi2.ppf(d['lambdaBSPS']['confid_lev'], 3)
    d['lambdaBSPS']['SD']                          = math.sqrt(d['lambdaBSPS']['dchi2'])
    # d['lambdaBSPS']['mesh']                        = get_meshes(ee_d, data_folder)
    d['lambdaBSPS']['covariance_epicenter_volume'] = get_cov_volume(ee_d['PosCovMat_3d'], d['lambdaBSPS']['SD'])
    d['lambdaBSPS']['npts_mw']                     = get_npts_mw(d['lambdaBSPS']['covariance_epicenter_volume'])
    d['lambdaBSPS']['gaussian_ellipsoid']          = get_gaussian_ellipsoid_3d(ee_d, ee_d['PosCovMat_3d'], d['lambdaBSPS']['SD'], d['lambdaBSPS']['npts_mw'])
    d['lambdaBSPS']                                = get_gaussian_ellipsoid_tetraedons(d['lambdaBSPS'], ee_d)

    return d['lambdaBSPS']

def get_cov_volume(cov_matrix, std):

    w, v = np.linalg.eig(cov_matrix)
    l_major = std*np.sqrt(w[0]) * 1000.0
    l_inter = std*np.sqrt(w[1]) * 1000.0
    l_minor = std*np.sqrt(w[2]) * 1000.0

    volume = (4./3.) * np.pi * l_major * l_inter * l_minor

    return volume

def get_npts_mw(volume):
    """
    Calculate the number of points to define de ellipsoide.
    Fitting function a*x**b found by F.Romano
    """

    a         = 0.6211
    b         = 0.4358
    n_tetra   = 23382 
    vol_tetra = 3.9990e+15

    npts_mw   = np.ceil(a*(volume*n_tetra/vol_tetra)**b).astype(int)
    npts_mw   = max(10, npts_mw)

    return npts_mw

def get_gaussian_ellipsoid_3d(ee, cov, std, npts): 

    center = [ee['ee_utm'][0], ee['ee_utm'][1], ee['depth']*-1000.0]

    cov = cov*1e6
    w, v = np.linalg.eigh(cov)
    if np.any(w < 0):
        print('Warning: negative eigenvalues')
        w = max(w,0)
    w = std * np.sqrt(w)    #get std of the cov matrix

    volume = (4./3.) * np.pi * w[0] * w[1] * w[2]

    # Make 3x 11x11 arrays
    x, y, z = create_sphere(npts)

    x = np.transpose(x)
    y = np.transpose(y)
    z = np.transpose(z)

    # Flattern 11x11 array
    ap = np.array([np.ravel(x), np.ravel(y), np.ravel(z)])

    bp = np.dot(np.dot(v, np.diag(w)), ap) +  \
         np.transpose(np.tile(center, (np.shape(ap)[1], 1)))

    xp = np.reshape(bp[0, :], np.shape(x))
    yp = np.reshape(bp[1, :], np.shape(y))
    zp = np.reshape(bp[2, :], np.shape(z))

    ellipsoid = {'xp':xp, 'yp':yp, 'zp':zp, 'vol': volume}
    print(" --> Volume of the Gaussian Ellipsoid: {:.8e} [m^3]".format(volume))

    return ellipsoid

def create_sphere(n_points=None, radius=None):
    """
    Create a discrete 3D spheric surface (points)
    Reference to create the shere:
       https://it.mathworks.com/matlabcentral/answers/48240-surface-of-a-equation:
       n = 100;
        r = 1.5;
        theta = (-n:2:n)/n*pi;
        phi = (-n:2:n)'/n*pi/2;
        cosphi = cos(phi); cosphi(1) = 0; cosphi(n+1) = 0;
        sintheta = sin(theta); sintheta(1) = 0; sintheta(n+1) = 0;
        x = r*cosphi*cos(theta);
        y = r*cosphi*sintheta;
        z = r*sin(phi)*ones(1,n+1);
        surf(x,y,z)
        xlabel('X'); ylabel('Y'); zlabel('Z')
    """
    if radius is None:
        radius = 1.0

    if n_points is None:
        n_points = 20

    theta = np.matrix(np.arange(-1*n_points,n_points+1,2) / n_points * np.pi)
    phi   = np.matrix(np.arange(-1*n_points,n_points+1,2) / n_points * np.pi / 2)
    phi   = phi.transpose()

    X = radius*np.matmul(np.cos(phi),np.cos(theta))
    Y = radius*np.matmul(np.cos(phi),np.sin(theta))
    Z = radius*np.matmul(np.sin(phi),np.matrix(np.ones(11)))

    # Set to 0 the very small numbers
    X[0] = 0
    X[-1] = 0
    Y[0] = 0
    Y[-1] = 0
    Y[:,0] = 0
    Y[:,-1] = 0

    return X,Y,Z

def get_gaussian_ellipsoid_tetraedons(el, ee):
    """
    From R. Tonini
    """


    xp = el['gaussian_ellipsoid']['xp']
    yp = el['gaussian_ellipsoid']['yp']
    zp = el['gaussian_ellipsoid']['zp'] #*-1.0

    # array preparation for creating tetrahedrons
    # This is like sss on matptf
    sss = np.vstack([xp.flatten(), yp.flatten(), zp.flatten()]).transpose()
    points_xyz = np.unique(sss, axis=0)
    n_points, tmp = np.shape(points_xyz)

    # lon lat conversion
    points_ll = np.zeros((n_points, 3))
    for i in range(n_points):
        points_ll[i, [1, 0]] = utm.to_latlon(points_xyz[i, 0],
                                             points_xyz[i, 1],
                                             ee['ee_utm'][2], ee['ee_utm'][3])

    points_ll[:, 2] = points_xyz[:, 2]

    # Good but slower (0.0030319690704345703 <=> 0.0013072490692138672)
    # from pyhull.delaunay import DelaunayTri
    # tetrahedrons = np.asarray(DelaunayTri(points_ll).vertices)
    # tetrahedron discretization (based on the points on the surface)
    tessellation = scipy.spatial.Delaunay(points_ll)
    tetrahedrons = tessellation.simplices

    # computing barycenters
    tetra_bar          = {}
    tetra_bar["utm_x"] = np.mean(points_xyz[tetrahedrons, 0], axis=1)
    tetra_bar["utm_y"] = np.mean(points_xyz[tetrahedrons, 1], axis=1)
    tetra_bar["lon"]   = np.mean(points_ll[tetrahedrons, 0], axis=1)
    tetra_bar["lat"]   = np.mean(points_ll[tetrahedrons, 1], axis=1)
    tetra_bar["depth"] = np.mean(points_ll[tetrahedrons, 2], axis=1)

    tetra_xyz = np.column_stack((tetra_bar["utm_x"],
                                 tetra_bar["utm_y"],
                                 tetra_bar["depth"]))

    n_tetra = len(tetra_bar["lon"])
    print(" --> N. Tetra in the Gaussian Ellipsoid: {0}".format(n_tetra))

    # computing tetrahedrons volume
    volume = np.zeros((n_tetra))
    for i in range(n_tetra):
        mm = np.column_stack((points_xyz[tetrahedrons[i, :], :],
                              np.array([1, 1, 1, 1])))
        volume[i] = np.abs(np.linalg.det(mm)/6.)

    volume_tot = np.sum(volume)
    print(" --> Volume of Tetra in the Gaussian Ellipsoid: %.8e [m^3]" % volume_tot)

    Vol_diff_perc = (el['gaussian_ellipsoid']['vol'] - volume_tot) / el['gaussian_ellipsoid']['vol']*100
    print(" --> Volume difference Gaussian <--> Tetra: %.2f [%%]" % Vol_diff_perc)

    el['tetra_bar']                 = tetra_bar
    el['tetrahedrons']              = tetrahedrons
    el['gaussian_ellipsoid_volume'] = volume_tot
    el['volumes_elements']          = volume
    el['tetra_xyz']                 = tetra_xyz

    return el    

In [ ]:
# These functions are copied from the step3_run.py file

def compute_hazard_curves(mih, prob_scenarios, n_pois, thresholds, sigma):

    n_thr = len(thresholds)

    hazard_curves_pois = np.zeros((n_pois, n_thr))

    # print(n_pois, n_thr, n_scen)
    # print(mih.shape, prob_scenarios.shape)

    # hazard_mode = 'lognormal'
    for ip in range(n_pois):

            mih_at_poi = mih[:,ip]
            ind_tmp = np.array(mih_at_poi == 0)
            mih_at_poi[ind_tmp] = 1.e-12

            mu = mih_at_poi
            mu = mu.reshape(len(mu), 1)

            cond_hazard_curve_tmp = 1 - scipy.stats.lognorm.cdf(thresholds, sigma, scale=mu).transpose()
            hazard_curves_pois[ip,:] = np.sum(prob_scenarios*cond_hazard_curve_tmp, axis=1)

    # # hazard_mode = 'lognormal_v1'
    # mih_coo = scipy.sparse.coo_array(mih)
    # print("Number of non-zero elements = {}".format(len(mih_coo.data)))
    # df_mihs = pd.DataFrame({"id_scen":mih_coo.row,"id_poi":mih_coo.col,"mih_value":mih_coo.data})
    # df_prob_scenarios = pd.DataFrame(prob_scenarios)\
    #                             .reset_index()\
    #                             .rename(columns={'index':'id_scen',0:'prob_scen'})
    # df_mihs = df_mihs.merge(df_prob_scenarios,how='left',left_on='id_scen', right_on='id_scen')

    # for ith,threshold in enumerate(thresholds[:]):
    #     df_mihs_thr = df_mihs.copy(deep=True)
    #     col_name = 'prob_lognorm_{}'.format(threshold)
    #     df_mihs_thr[col_name] = 1-scipy.stats.lognorm.cdf(threshold, sigma, scale=df_mihs_thr['mih_value']).transpose()
    #     df_mihs_thr[col_name] = df_mihs_thr[col_name]*df_mihs_thr['prob_scen']
    #     df_mihs_thr = df_mihs_thr.groupby(by='id_poi').agg({col_name:'sum'})
    #     hazard_curves_pois[df_mihs_thr.index,ith]=df_mihs_thr[col_name].to_numpy()

    return hazard_curves_pois

In [ ]:
# These functions are copied from the step5_run.py file

def plot_hazard_maps(points, hmaps, event_dict, map_label):
    """
    """

    proj = cartopy.crs.PlateCarree()
    #cmap = plt.cm.magma_r
    cmap = plt.cm.jet
    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']
    ev_depth = event_dict['depth']
    ev_mag = event_dict['mag']
    ev_place = event_dict['place']

    for key, hmap in hmaps.items():           
                
        print("mapping ... {}".format(key))
        fig = plt.figure(figsize=(16, 8))
        # ax = plt.axes(projection=cartopy.crs.Mercator())
        usemap_proj = cartopy.crs.PlateCarree(central_longitude=180)
        ax = plt.axes(projection=usemap_proj)
        coastline = cartopy.feature.GSHHSFeature(scale='low', levels=[1])
        #coastline = cartopy.feature.GSHHSFeature(scale='high', levels=[1])
        ax.add_feature(coastline, edgecolor='#000000', facecolor='#cccccc', linewidth=1)
        ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'))
        ax.add_feature(cartopy.feature.STATES.with_scale('50m'))
        ax.add_feature(cartopy.feature.OCEAN.with_scale('50m'))
        gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1,
                           color="#ffffff", alpha=0.5, linestyle='-')
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True
        gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
        gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
        gl.ylabel_style = {'size': 14}

        sc = ax.scatter(points[:,0], points[:,1], c=hmap, s=17, marker="o", 
                    linewidths=0.75, edgecolors="#000000", label=" ",
                    cmap=cmap, clip_on=True,# vmin=map_min, vmax=map_max, 
                    transform=proj, zorder=10, norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=50))#(vmin=map_min, vmax=map_max))

        ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=14, 
                #markerfacecolor='#c0bfbc', markeredgecolor='#000000', 
                markerfacecolor='white', markeredgecolor='#000000', 
                transform=proj)

        cbar = plt.colorbar(sc, shrink=0.75)
        #cbar.ax.set_yticklabels(labels=cbar.ax.get_yticklabels(), fontsize=10)
        cbar.set_label(label=f'(m)', size=12)
        # ax.set_title("Hazard map - {0}".format(key))
        plt.suptitle("Hazard map - {0}".format(key),fontsize=24)
        plt.title("Epicentral Region: {0} \n Event parameters: Lon={1}, Lat={2}, Depth={3}; Magnitude={4}".format(ev_place, ev_lon, ev_lat, ev_depth, str(ev_mag)[:3] ),fontsize=18)
        ax.set_xlabel(r'Longitude ($^\circ$)', fontsize=14)
        ax.set_ylabel(r'Latitude ($^\circ$)', fontsize=14)